# Projeto 1 - Ciência dos Dados

Nome: Alexia Barbosa Pires

Nome: Ellen Coutinho Lião da Silva

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import re
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Alexia\Documents\2° semestre 22.2\CDados\Projetos\p1_alexia_ellen\22-2a-cd-p1-grupo_alexiabp\notebooks


Carregando a base de dados com as notícias classificadas manualmente:

In [3]:
path = pathlib.Path(r"..\data")
filename = f'{path}\dados.xlsx'

In [4]:
#Base de TREINAMENTO:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Categoria,Titulo,Descrição,Data,Pagina,Target
0,financas,CORREÇÃO-Dólar caminha para 4ª alta semanal se...,(Corrige no 9º parágrafo para “outubro” no lug...,13/05/2022 12:04,71,1
1,financas,"Ouro fecha em alta, após quedas recentes, com ...",O contrato mais líquido do ouro fechou em alta...,03/05/2022 15:16,78,2
2,financas,"Bolsas da Europa fecham em queda, sem resoluçã...",As principais bolsas europeias fecharam em que...,10/03/2022 14:26,110,0
3,economia,"Aneel aprova reajuste médio de 15,77% nas tari...",A Agência Nacional de Energia Elétrica (Aneel)...,23/08/2022 11:21,57,0
4,financas,Dólar reduz queda com fortalecimento do DXY ap...,O dólar reduziu a queda intradia frente ao rea...,18/08/2022 10:33,12,1


In [5]:
#BASE DE TESTES:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

C:\Users\Alexia\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\Alexia\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Categoria,Titulo,Descrição,Data,Pagina,Target
0,economia,Atividade econômica do País está acima do espe...,O diretor de Política Econômica do Banco Centr...,11/07/2022 12:57,212,0
1,economia,Tire suas dúvidas sobre a vacina contra a varí...,Um lote da vacina contra a varíola dos macacos...,17/08/2022 21:20,73,0
2,financas,Taxas futuras de juros recuam com ajustes apes...,As taxas de juros negociadas no mercado futuro...,15/07/2022 11:10,32,0
3,economia,Corte de Nova York permite que Harvey Weinstei...,Por Daniel Trotta (Reuters) – A mais alta cort...,25/08/2022 11:00,48,0
4,financas,"Bolsas de NY fecham em baixa, com guerra na Uc...",As bolsas de Nova York fecharam em baixa nesta...,03/03/2022 18:26,115,0


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação das notícias (Target).

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

### 1°) Vamos fazer uma limpeza no título e na descrição de cada ntícia com a target = 0
- Target 0 = neutro
- Target 1 = positivo
- Target 2 = negativo

In [6]:
def cleanup(text):
    #import string
    text = text.replace('[…]','')
    punctuation = '[!-.:?;\/–]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [7]:
#Juntando Título e descrição do target 0
target_0 = test.loc[train['Target']==0,:]
string_0 = ''
for texto in target_0['Descrição']:
    novo_texto=cleanup(str(texto).lower())
    string_0+=''.join(novo_texto)
    
for titulo in target_0['Titulo']:
    novo_titulo=cleanup(str(titulo).lower())
    string_0+=''.join(novo_titulo)
    
    
lista_0 = string_0.split()

In [8]:
#Juntando Título e descrição do target 1
target_1 = test.loc[train['Target']==1,:]
string_1 = ''
for texto in target_1['Descrição']:
    novo_texto=cleanup(str(texto).lower())
    string_1+=''.join(novo_texto)
    
for titulo in target_1['Titulo']:
    novo_titulo=cleanup(str(titulo).lower())
    string_1+=''.join(novo_titulo)
    
    
lista_1 =string_1.split()

In [9]:
#Juntando Título e Descrição do Target 2
target_2 = test.loc[train['Target']==2,:]
string_2 = ''
for texto in target_2['Descrição']:
    novo_texto=cleanup(str(texto).lower())
    string_2+=''.join(novo_texto)
    
for titulo in target_2['Titulo']:
    novo_titulo=cleanup(str(titulo).lower())
    string_2+=''.join(novo_titulo)
    
    
lista_2 = string_2.split()

### 2°) Agora, faremos uma tabela de frequencias relativas e absolutas para cada stringzona! Queremos a frequência de cada palavra

### Cálculo da frequência para target NEUTRA

In [10]:
serie_0 = pd.Series(string_0.split())
tabela_0 = serie_0.value_counts(True)
tabela_0

de         0.057263
a          0.032068
o          0.031525
em         0.026040
do         0.023568
             ...   
vistos     0.000060
pe         0.000060
dentre     0.000060
espiral    0.000060
gates      0.000060
Length: 3854, dtype: float64

In [11]:
#Frequencia absoluta para Serie  0
tabela_abs_0 = serie_0.value_counts()
tabela_abs_0

de         950
a          532
o          523
em         432
do         391
          ... 
vistos       1
pe           1
dentre       1
espiral      1
gates        1
Length: 3854, dtype: int64

### Cálculo da frequencia para targets 1 

In [12]:
serie_1 = pd.Series(string_1.split())
tabela_1 = serie_1.value_counts(True)
tabela_1

de        0.058670
o         0.033898
a         0.030639
em        0.029335
do        0.026076
            ...   
esboço    0.000652
otan      0.000652
98        0.000652
ny        0.000652
doses     0.000652
Length: 705, dtype: float64

In [13]:
#Frequencia absoluta para Serie  0
tabela_abs_1 = serie_1.value_counts()
tabela_abs_1

de        90
o         52
a         47
em        45
do        40
          ..
esboço     1
otan       1
98         1
ny         1
doses      1
Length: 705, dtype: int64

### Cálculo da frequencia para targets 2

In [14]:
serie_2 = pd.Series(string_2.split())
tabela_2 = serie_2.value_counts(True)
tabela_2

de               0.060514
a                0.037821
em               0.033283
o                0.026223
e                0.020171
                   ...   
grave            0.000504
operou           0.000504
jurisdição       0.000504
payrollcaixas    0.000504
aliança          0.000504
Length: 857, dtype: float64

In [15]:
#Frequencia absoluta para Serie  2
tabela_abs_2 = serie_2.value_counts()
tabela_abs_2

de               120
a                 75
em                66
o                 52
e                 40
                ... 
grave              1
operou             1
jurisdição         1
payrollcaixas      1
aliança            1
Length: 857, dtype: int64

Para fins de comparação, é necessário também unir todas as palavras das notícias em uma única string e montar as tabelas de frequências

In [16]:
todas = string_0+string_1+string_2
lista_todas=todas.split()
serie_todas = pd.Series(lista_todas)

tabela_todas = serie_todas.value_counts(True)
tabela_abs_todas = serie_todas.value_counts()
tabela_abs_todas



de              1160
a                654
o                626
em               543
do               463
                ... 
certa              1
governos           1
descontentes       1
plenário           1
gates              1
Length: 4355, dtype: int64

## Montando o classificador

In [17]:
prob_neutra = len(serie_0)/len(serie_todas)
prob_positiva = len(serie_1)/len(serie_todas)
prob_negativa= len(serie_2)/len(serie_todas)

In [18]:
#P(palavra|target)
#noticia=cleanup(str(texto).lower())

test['Junção']=test['Titulo']+' '+test['Descrição']
i=0
for noticia in test['Junção']:
    #print(noticia)
    #print(' ')
    nova_noticia = str(noticia).lower()
    nova_noticia = cleanup(nova_noticia)
    
    #print(nova_noticia)
    test['Junção'][i]=nova_noticia
    i+=1
    
test.head()

<ipython-input-18-a5da9702b3da>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Junção'][i]=nova_noticia


,Categoria,Titulo,Descrição,Data,Pagina,Target,Junção
0,economia,Atividade econômica do País está acima do espe...,O diretor de Política Econômica do Banco Centr...,11/07/2022 12:57,212,0,atividade econômica do país está acima do espe...
1,economia,Tire suas dúvidas sobre a vacina contra a varí...,Um lote da vacina contra a varíola dos macacos...,17/08/2022 21:20,73,0,tire suas dúvidas sobre a vacina contra a varí...
2,financas,Taxas futuras de juros recuam com ajustes apes...,As taxas de juros negociadas no mercado futuro...,15/07/2022 11:10,32,0,taxas futuras de juros recuam com ajustes apes...
3,economia,Corte de Nova York permite que Harvey Weinstei...,Por Daniel Trotta (Reuters) – A mais alta cort...,25/08/2022 11:00,48,0,corte de nova york permite que harvey weinstei...
4,financas,"Bolsas de NY fecham em baixa, com guerra na Uc...",As bolsas de Nova York fecharam em baixa nesta...,03/03/2022 18:26,115,0,bolsas de ny fecham em baixa com guerra na ucr...


In [23]:
#Aqui, separamos cada probabilidade em listas vazias para guardar as probabilidades condicionais de cada target.
probNeutra = []
probPos = []
probNeg = []

#Fazemos esse for pra calcular as probabilidades condicionais de acordo com cada junção = (Titulo + Descrição de cada linha)
#Além disso, usamos a aba de "Testes" pois queremos que o nosso classificador saiba como julgar cada notícia.
for noticia in test['Junção']:
    separacao = noticia.split()
    
    probJuncao_dado_Neutra = 1
    probJuncao_dado_Pos = 1
    probJuncao_dado_Neg = 1
    
    for palavra in separacao:
        #Partir daqui
        #Se a palavra estiver na tabela com freqabs de notc neutras:
        #P(Palabra|FreqANeutra) = P(FreqNeutra|Palavra)*P(Palavra)/P(FreqAbsNeutra)
        if palavra in tabela_abs_0:
            probJuncao_dado_Neutra *= (tabela_abs_0[palavra]+1) / (tabela_abs_0.sum() + len(tabela_abs_todas))
        else:
            probJuncao_dado_Neutra *= (0+1)/(tabela_abs_0.sum() + len(tabela_abs_todas))
            
        #Se a palav estiver na tabela c freq abs de notc positiva:
        if palavra in tabela_abs_1:
            probJuncao_dado_Pos *= (tabela_abs_1[palavra]+1) / (tabela_abs_1.sum() + len(tabela_abs_todas))
        else:
            probJuncao_dado_Pos*= (0+1)/(tabela_abs_1.sum() + len(tabela_abs_todas))
            
        #Se a palavr estiver na tabela c freqabs de notic negativa:
        if palavra in tabela_abs_2:
            probJuncao_dado_Neg *= (tabela_abs_2[palavra]+1) / (tabela_abs_2.sum() + len(tabela_abs_todas))
        else:
            probJuncao_dado_Neg *= (0+1)/(tabela_abs_2.sum() + len(tabela_abs_todas))
    probNeutra.append(probJuncao_dado_Neutra)
    probPos.append(probJuncao_dado_Pos)
    probNeg.append(probJuncao_dado_Neg)

        

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**